In [ ]:
import pandas as pd
import numpy as np
import json
import plotly.express as px
import sqlite3

from utils import notebook_util
notebook_util.disp_notebook_full_width()

In [ ]:
# from jacob; grabs AA sequence

def get_prot_sequence(pid):
    con=sqlite3.connect("/GeneGraphDB/data/20220322_80kprotein_stats.db")
    cur = con.cursor()
    cmd = "SELECT sequence FROM proteins WHERE pid = '%s'" % pid
    cur.execute(cmd)
    return str(cur.fetchone()[0])
    con.close()

In [ ]:
with open("/GeneGraphDB/data/20220308_neo4j_colocalization/cas1.json") as f:
    cas1_dict = json.load(f)

In [ ]:
cas1_df = pd.DataFrame(cas1_dict.values())
cas1_df.head()

In [ ]:
get_prot_sequence("c0c421cd2905bef4bb")

In [ ]:
cas1_df['weighted_score'] = cas1_df['tgt_colocalization'] * np.power(cas1_df['num_tgt_p100s'], 1/3)

In [ ]:
# cas1_df = pd.DataFrame(cas1_dict.values())
print(len(cas1_df), "rows total")
cas1_df = cas1_df[cas1_df['num_tgt_p100s'] >= 5]
print(len(cas1_df), "filtered for >5 tgt p100")
cas1_df = cas1_df[cas1_df['num_connections'] > 1]
print(len(cas1_df), "+ filtered for >1 shared edges")

cas1_df = cas1_df[cas1_df['tgt_colocalization'] > 0.1]
print(len(cas1_df), "+ filtered for >0.1 tgt colocalization")

cas1_df = cas1_df[cas1_df['bait_colocalization'] > 0.01]
print(len(cas1_df), "+ filtered for >0.01 bait colocalization")


cas1_df.sort_values('weighted_score', ascending=False, inplace=True)
cas1_df
# cas1_df.sort_values('tgt_colocalization', ascending=False)

In [ ]:
for tgt_p30 in cas1_df['tgt_p30'][0:10]:
    print(tgt_p30)
    print()
    print(get_prot_sequence(tgt_p30))
    display(cas1_df[(cas1_df['tgt_p30'] == tgt_p30) | (cas1_df['bait_p30'] == tgt_p30)])
    print("\n\n*********\n")

In [ ]:
fig = px.histogram(cas1_df[cas1_df["num_tgt_p100s"] >= 5], x="num_tgt_p100s")
fig.show()

In [ ]:
cas1_df = cas1_df[cas1_df["num_tgt_p100s"] >= 5]